In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

/Users/muddy_flesh/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def get_bound(index1, index2, bonds):
    bond = frozenset({tuple(index1),tuple(index2)})
    if bond in bonds:
        return bonds[bond]
    else:
        bonds[bond] = np.random.standard_normal()
        return bonds[bond]

In [4]:
def increase_entry_by_one(A, j):
    A_modified = A.copy()
    A_modified[j] += 1
    return A_modified
def decrease_entry_by_one(A, j):
    A_modified = A.copy()
    A_modified[j] -= 1
    return A_modified

In [5]:
def get_neighbor(indices,L):
    neighbor_index = []
    for j in range(len(L)):
        if (indices[j] == 0):
            neighbor_index.append(increase_entry_by_one(indices, j))
            indice_copy = indices.copy()
            indice_copy[j] = L[j]-1
            neighbor_index.append(indice_copy)
        elif (indices[j] == L[j]-1):
            neighbor_index.append(decrease_entry_by_one(indices, j))
            indice_copy = indices.copy()
            indice_copy[j] = 0
            neighbor_index.append(indice_copy)
        else:
            neighbor_index.append(increase_entry_by_one(indices, j))
            neighbor_index.append(decrease_entry_by_one(indices, j))
    return neighbor_index

In [6]:
def get_energy(spin, spin_index, neighbor_index, S):
    energy = 0
    for neighbor in neighbor_index:
        bond = get_bound(spin_index, neighbor, bonds)
        energy = energy + bond*spin*S[tuple(neighbor)]
    return energy

In [7]:
def overlap(S1,S2,N):
    return np.sum(S1*S2)/N

def sweep(S,L,N):
    sweep = 0
    while sweep < N:
        indices = [np.random.choice(dim) for dim in L]
        spin = S[tuple(indices)]
        neighbor_index = get_neighbor(indices,L)
        beforeE = get_energy(spin, indices, neighbor_index, S)
        afterE = get_energy(-spin, indices, neighbor_index, S)
        deltaE = afterE - beforeE
        if deltaE > 0:
            S[tuple(indices)] = -spin
        sweep = sweep+1

In [8]:
def is_active(index, S, L):
    spin = S[tuple(index)]
    neighbor_index = get_neighbor(np.asarray(index),L)
    beforeE = get_energy(spin, index, neighbor_index, S)
    afterE = get_energy(-spin, index, neighbor_index, S)
    deltaE = afterE - beforeE
    return deltaE > 0

In [9]:
def get_active(S,L):
    it = np.nditer(S,flags = ['multi_index'])
    active_indices = []
    while not it.finished:
        index = it.multi_index
        if is_active(index, S, L):
            active_indices.append(index)
        it.iternext()
    return active_indices

In [10]:
import random

def kineticMonteCarlo(S,L,active_list):
    l = len(active_list)
    if l == 0:
        return 0
    t = 1/l
    index = random.choice(active_list)
    spin = S[tuple(index)]
    neighbor_index = get_neighbor(np.asarray(index),L)
    beforeE = get_energy(spin, index, neighbor_index, S)
    afterE = get_energy(-spin, index, neighbor_index, S)
    deltaE = afterE - beforeE
    if deltaE > 0:
        S[tuple(index)] = -spin
        active_list.remove(tuple(index))
        for nspin in neighbor_index:
            if is_active(nspin,S,L):
                if not (tuple(nspin) in active_list):
                    active_list.append(tuple(nspin))
            else:
                if (tuple(nspin) in active_list):
                    active_list.remove(tuple(nspin))
    return t

In [31]:
bonds = dict()
L = np.array([100, 100])
N = np.prod(L)
model = 'EA'
config = np.random.choice([-1, 1], size=tuple(L))
S2 = config.copy()
S1 = config.copy()

In [32]:
survival1 = 0
survival2 = 0

while len(get_active(S1,L)) != 0:
    sweep(S1,L,N)
    survival1 = survival1 + 1
    print('Sweep {} done'.format(survival1))
    if survival1 == 10:
        break

S1_active = get_active(S1,L)

while True:
    k = kineticMonteCarlo(S1,L,S1_active)
    if k == 0:
        break
    survival1 = survival1 + k

while len(get_active(S2,L)) != 0:
    sweep(S2,L,N)
    survival2 = survival2 + 1
    print('Sweep {} done'.format(survival2))
    if survival2 == 10:
        break

S2_active = get_active(S2,L)

while True:
    k = kineticMonteCarlo(S2,L,S2_active)
    if k == 0:
        break
    survival2 = survival2 + k

print(overlap(S1,S2,N))
print(survival1)
print(survival2)   

Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
0.4122
11.0
11.5


In [33]:
bonds = dict()
L = np.array([100, 35, 35])
N = np.prod(L)
model = 'EA'
config = np.random.choice([-1, 1], size=tuple(L))
S2 = config.copy()
S1 = config.copy()

survival1 = 0
survival2 = 0

while len(get_active(S1,L)) != 0:
    sweep(S1,L,N)
    survival1 = survival1 + 1
    print('Sweep {} done'.format(survival1))
    if survival1 == 10:
        break

S1_active = get_active(S1,L)

while True:
    k = kineticMonteCarlo(S1,L,S1_active)
    if k == 0:
        break
    survival1 = survival1 + k

while len(get_active(S2,L)) != 0:
    sweep(S2,L,N)
    survival2 = survival2 + 1
    print('Sweep {} done'.format(survival2))
    if survival2 == 10:
        break

S2_active = get_active(S2,L)

while True:
    k = kineticMonteCarlo(S2,L,S2_active)
    if k == 0:
        break
    survival2 = survival2 + k

print(overlap(S1,S2,N))
print(survival1)
print(survival2) 

Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
0.3580081632653061
17.200353336376395
18.83392539721801


In [34]:
bonds = dict()
L = np.array([100, 10, 10, 10])
N = np.prod(L)
model = 'EA'
config = np.random.choice([-1, 1], size=tuple(L))
S2 = config.copy()
S1 = config.copy()

survival1 = 0
survival2 = 0

while len(get_active(S1,L)) != 0:
    sweep(S1,L,N)
    survival1 = survival1 + 1
    print('Sweep {} done'.format(survival1))
    if survival1 == 10:
        break

S1_active = get_active(S1,L)

while True:
    k = kineticMonteCarlo(S1,L,S1_active)
    if k == 0:
        break
    survival1 = survival1 + k

while len(get_active(S2,L)) != 0:
    sweep(S2,L,N)
    survival2 = survival2 + 1
    print('Sweep {} done'.format(survival2))
    if survival2 == 10:
        break

S2_active = get_active(S2,L)

while True:
    k = kineticMonteCarlo(S2,L,S2_active)
    if k == 0:
        break
    survival2 = survival2 + k

print(overlap(S1,S2,N))
print(survival1)
print(survival2) 

Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
0.32598
22.40991434512194
19.781636978116094


In [35]:
bonds = dict()
L = np.array([100, 6, 6, 6, 6])
N = np.prod(L)
model = 'EA'
config = np.random.choice([-1, 1], size=tuple(L))
S2 = config.copy()
S1 = config.copy()

survival1 = 0
survival2 = 0

while len(get_active(S1,L)) != 0:
    sweep(S1,L,N)
    survival1 = survival1 + 1
    print('Sweep {} done'.format(survival1))
    if survival1 == 10:
        break

S1_active = get_active(S1,L)

while True:
    k = kineticMonteCarlo(S1,L,S1_active)
    if k == 0:
        break
    survival1 = survival1 + k

while len(get_active(S2,L)) != 0:
    sweep(S2,L,N)
    survival2 = survival2 + 1
    print('Sweep {} done'.format(survival2))
    if survival2 == 10:
        break

S2_active = get_active(S2,L)

while True:
    k = kineticMonteCarlo(S2,L,S2_active)
    if k == 0:
        break
    survival2 = survival2 + k

print(overlap(S1,S2,N))
print(survival1)
print(survival2) 

Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
0.29878086419753086
21.16139052794436
22.42726034690511


In [36]:
bonds = dict()
L = np.array([100, 4, 4, 4, 4, 4])
N = np.prod(L)
model = 'EA'
config = np.random.choice([-1, 1], size=tuple(L))
S2 = config.copy()
S1 = config.copy()

survival1 = 0
survival2 = 0

while len(get_active(S1,L)) != 0:
    sweep(S1,L,N)
    survival1 = survival1 + 1
    print('Sweep {} done'.format(survival1))
    if survival1 == 10:
        break

S1_active = get_active(S1,L)

while True:
    k = kineticMonteCarlo(S1,L,S1_active)
    if k == 0:
        break
    survival1 = survival1 + k

while len(get_active(S2,L)) != 0:
    sweep(S2,L,N)
    survival2 = survival2 + 1
    print('Sweep {} done'.format(survival2))
    if survival2 == 10:
        break

S2_active = get_active(S2,L)

while True:
    k = kineticMonteCarlo(S2,L,S2_active)
    if k == 0:
        break
    survival2 = survival2 + k

print(overlap(S1,S2,N))
print(survival1)
print(survival2) 

Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
0.28375
24.25855650590691
21.940183082113823


In [37]:
bonds = dict()
L = np.array([100, 3, 3, 3, 3, 3, 3])
N = np.prod(L)
model = 'EA'
config = np.random.choice([-1, 1], size=tuple(L))
S2 = config.copy()
S1 = config.copy()

survival1 = 0
survival2 = 0

while len(get_active(S1,L)) != 0:
    sweep(S1,L,N)
    survival1 = survival1 + 1
    print('Sweep {} done'.format(survival1))
    if survival1 == 10:
        break

S1_active = get_active(S1,L)

while True:
    k = kineticMonteCarlo(S1,L,S1_active)
    if k == 0:
        break
    survival1 = survival1 + k

while len(get_active(S2,L)) != 0:
    sweep(S2,L,N)
    survival2 = survival2 + 1
    print('Sweep {} done'.format(survival2))
    if survival2 == 10:
        break

S2_active = get_active(S2,L)

while True:
    k = kineticMonteCarlo(S2,L,S2_active)
    if k == 0:
        break
    survival2 = survival2 + k

print(overlap(S1,S2,N))
print(survival1)
print(survival2) 

Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
Sweep 1 done
Sweep 2 done
Sweep 3 done
Sweep 4 done
Sweep 5 done
Sweep 6 done
Sweep 7 done
Sweep 8 done
Sweep 9 done
Sweep 10 done
0.2878189300411523
26.445836665395746
24.085595613574284
